# Emotion Companion - Local Demo

This notebook demonstrates how to use the Emotion Companion NLP analysis locally without running the full application.

## Setup

Make sure you have installed all dependencies:
```bash
pip install -r requirements.txt
python -m spacy download en_core_web_sm
```

In [ ]:
# Import required libraries
import sys
import os
import pandas as pd
import plotly.express as px
import json

# Add parent directory to path
sys.path.append('..')

from backend.nlp import analyze_text, sentiment, emotion, extract_themes

## 1. Analyze a Single Text Entry

In [ ]:
# Sample journal entry
sample_text = """
I'm feeling really anxious about my upcoming presentation at work tomorrow. 
I've prepared well, but I can't shake this nervous feeling. At the same time, 
I'm excited about the opportunity to showcase my project to the team.
"""

# Perform analysis
result = analyze_text(sample_text)

# Display results
print("=" * 60)
print("EMOTION ANALYSIS RESULTS")
print("=" * 60)
print(f"\nText: {sample_text.strip()}")
print(f"\n📊 Sentiment: {result['sentiment']['label']} (score: {result['sentiment']['score']})")
print(f"\n😊 Primary Emotion: {result['emotion']['primary_emotion']} {result['emotion']['emoji']}")
print(f"\n🎯 Mood Score: {result['mood_score']}/10")
print(f"\n🏷️ Themes: {', '.join(result['themes'])}")
print(f"\n💡 Suggestions:")
for i, suggestion in enumerate(result['suggestions'], 1):
    print(f"   {i}. {suggestion}")

print(f"\n📈 All Emotion Scores:")
for emotion_name, score in result['emotion']['emotion_scores'].items():
    print(f"   {emotion_name}: {score:.3f}")

## 2. Analyze Sample Data from CSV

In [ ]:
# Load sample journal data
df = pd.read_csv('../sample_data/sample_journals.csv')
print(f"Loaded {len(df)} sample journal entries")
df.head()

In [ ]:
# Analyze all entries (this may take a minute)
print("Analyzing all entries...")

analyses = []
for idx, row in df.iterrows():
    analysis = analyze_text(row['text'])
    analyses.append({
        'text': row['text'][:50] + '...',
        'mood_score': analysis['mood_score'],
        'sentiment': analysis['sentiment']['label'],
        'sentiment_score': analysis['sentiment']['score'],
        'emotion': analysis['emotion']['primary_emotion'],
        'themes': ', '.join(analysis['themes'][:3])
    })

results_df = pd.DataFrame(analyses)
print("\nAnalysis complete!")
results_df

## 3. Visualize Results

In [ ]:
# Mood score distribution
fig = px.histogram(
    results_df, 
    x='mood_score',
    title='Mood Score Distribution',
    labels={'mood_score': 'Mood Score (0-10)'},
    nbins=11
)
fig.show()

In [ ]:
# Emotion distribution
emotion_counts = results_df['emotion'].value_counts()

fig = px.pie(
    values=emotion_counts.values,
    names=emotion_counts.index,
    title='Emotion Distribution'
)
fig.show()

In [ ]:
# Sentiment vs Mood Score
fig = px.scatter(
    results_df,
    x='sentiment_score',
    y='mood_score',
    color='emotion',
    title='Sentiment Score vs Mood Score',
    labels={
        'sentiment_score': 'Sentiment Score (-1 to 1)',
        'mood_score': 'Mood Score (0-10)'
    },
    hover_data=['text']
)
fig.show()

## 4. Test Individual NLP Components

In [ ]:
# Test sentiment analysis
test_texts = [
    "I am so happy and excited!",
    "I feel terrible and sad.",
    "Today was just okay."
]

print("Sentiment Analysis Tests:")
print("=" * 60)
for text in test_texts:
    result = sentiment(text)
    print(f"Text: {text}")
    print(f"Result: {result['label']} (score: {result['score']:.3f})")
    print()

In [ ]:
# Test emotion detection
test_texts = [
    "I'm feeling anxious and worried about tomorrow.",
    "I'm so angry and frustrated right now!",
    "I feel calm and peaceful."
]

print("Emotion Detection Tests:")
print("=" * 60)
for text in test_texts:
    result = emotion(text)
    print(f"Text: {text}")
    print(f"Primary Emotion: {result['primary_emotion']} {result['emoji']}")
    print(f"Top 3 Emotions:")
    sorted_emotions = sorted(
        result['emotion_scores'].items(),
        key=lambda x: x[1],
        reverse=True
    )[:3]
    for emotion_name, score in sorted_emotions:
        print(f"  - {emotion_name}: {score:.3f}")
    print()

In [ ]:
# Test theme extraction
test_text = """
I had a challenging day at work today. The project deadline is approaching,
and I'm feeling the pressure. My team has been supportive, but I'm worried
about meeting our goals. I need to find better work-life balance.
"""

themes = extract_themes(test_text)
print("Theme Extraction Test:")
print("=" * 60)
print(f"Text: {test_text.strip()}")
print(f"\nExtracted Themes:")
for i, theme in enumerate(themes, 1):
    print(f"  {i}. {theme}")

## 5. Export Results

In [ ]:
# Export analysis results to CSV
output_file = 'analysis_results.csv'
results_df.to_csv(output_file, index=False)
print(f"Results exported to {output_file}")

## 6. Model Information

In [ ]:
# Check which models are being used
from backend import nlp

print("NLP Model Status:")
print("=" * 60)
print(f"HuggingFace Sentiment Model: {'✓ Loaded' if nlp.sentiment_pipeline else '✗ Not loaded (using VADER)'}")
print(f"HuggingFace Emotion Model: {'✓ Loaded' if nlp.emotion_pipeline else '✗ Not loaded (using keywords)'}")
print(f"VADER Analyzer: {'✓ Loaded' if nlp.vader_analyzer else '✗ Not loaded'}")
print(f"spaCy Model: {'✓ Loaded' if nlp.nlp_spacy else '✗ Not loaded'}")

## Conclusion

This notebook demonstrated:
1. Analyzing individual text entries
2. Batch processing multiple entries
3. Visualizing emotional patterns
4. Testing individual NLP components
5. Exporting results

You can use this notebook to:
- Test the NLP pipeline locally
- Analyze your own journal data
- Experiment with different texts
- Understand how the analysis works

For the full application experience, run:
```bash
python -m backend.app  # Start backend
streamlit run streamlit_app/app.py  # Start frontend
```